In [9]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import pickle
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import log_loss, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import AdaBoostClassifier


In [4]:
# DC 
df = pd.read_csv("../data/train.csv").drop("id",axis=1)
df_test = pd.read_csv("../data/test.csv").drop("id",axis=1)

labels = df["target"].values
print(type(labels))
print(labels.shape)

unique_labels = np.unique(labels)
nb_labels = len(unique_labels)

df_train = df.drop("target", axis=1)

X = df_train.values
X_test = df_test.values

print(X.shape)
print(X_test.shape)

y = []
for label in labels:
    y.append(int(label[-1:])-1)
y = np.asarray(y)


print("Construction des centroïdes")
centroids = []
for label in unique_labels:
    same_labels = df_train.loc[df['target'] == label]
    same_labels = same_labels.values
    centroids.append(same_labels.sum(axis = 0)/same_labels.shape[0])
centroids = np.asarray(centroids)


print("Construction de la représentation dcDistance")

X_dc = pairwise_distances(X,centroids)
X_test_dc = pairwise_distances(X_test, centroids, metric='minkowski')

print("Construction terminée \n")

y_train = y

<class 'numpy.ndarray'>
(61878,)
(61878, 93)
(144368, 93)
Construction des centroïdes
Construction de la représentation dcDistance
Construction terminée 



In [5]:
print(X.shape)

(61878, 93)


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)

target_names = ['class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

In [7]:
print(X.shape)

(61878, 93)


In [10]:
clf = SVC(gamma=0.001, C=100)
clf.fit(X_train, y_train)
print("Fit terminé")
pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

Fit terminé


AttributeError: predict_proba is not available when  probability=False

In [11]:
#pred = clf.predict_proba(X_test)
pred_for_acc = clf.predict(X_test)
#print("%.2f" % log_loss(y_test, pred, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc, target_names=target_names))

             precision    recall  f1-score   support

    class_1       0.61      0.55      0.58       399
    class_2       0.70      0.87      0.78      3178
    class_3       0.61      0.42      0.50      1561
    class_4       0.77      0.44      0.56       538
    class_5       0.96      0.97      0.97       565
    class_6       0.94      0.94      0.94      2884
    class_7       0.74      0.65      0.69       552
    class_8       0.89      0.92      0.90      1674
    class_9       0.87      0.85      0.86      1025

avg / total       0.80      0.80      0.79     12376



In [12]:
clf_2 = SVC(gamma=0.001, C=100, probability=True)
clf_2.fit(X_train, y_train)
print("Fit terminé")
pred_2 = clf_2.predict_proba(X_test)
pred_for_acc_2 = clf_2.predict(X_test)
print("%.2f" % log_loss(y_test, pred_2, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc_2, target_names=target_names))

Fit terminé
0.52
             precision    recall  f1-score   support

    class_1       0.61      0.55      0.58       399
    class_2       0.70      0.87      0.78      3178
    class_3       0.61      0.42      0.50      1561
    class_4       0.77      0.44      0.56       538
    class_5       0.96      0.97      0.97       565
    class_6       0.94      0.94      0.94      2884
    class_7       0.74      0.65      0.69       552
    class_8       0.89      0.92      0.90      1674
    class_9       0.87      0.85      0.86      1025

avg / total       0.80      0.80      0.79     12376



In [16]:
clf_2 = SVC(gamma=0.001, C=100, probability=True, cache_size=4000, verbose=True, 
            class_weight='balanced')
clf_2.fit(X_train, y_train)
print("Fit terminé")
pred_2 = clf_2.predict_proba(X_test)
pred_for_acc_2 = clf_2.predict(X_test)
print("%.2f" % log_loss(y_test, pred_2, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc_2, target_names=target_names))

[LibSVM]Fit terminé
0.52
             precision    recall  f1-score   support

    class_1       0.48      0.70      0.57       399
    class_2       0.83      0.65      0.73      3178
    class_3       0.52      0.66      0.58      1561
    class_4       0.46      0.70      0.56       538
    class_5       0.96      0.98      0.97       565
    class_6       0.95      0.92      0.93      2884
    class_7       0.63      0.69      0.66       552
    class_8       0.91      0.88      0.90      1674
    class_9       0.87      0.82      0.84      1025

avg / total       0.80      0.78      0.78     12376



In [21]:
clf_2 = SVC(gamma=0.001, C=50, probability=True, cache_size=4000, verbose=True, max_iter=50)
clf_2.fit(X_train, y_train)
print("Fit terminé")
pred_2 = clf_2.predict_proba(X_test)
pred_for_acc_2 = clf_2.predict(X_test)
print("%.2f" % log_loss(y_test, pred_2, eps=1e-15, normalize=True))
print(classification_report(y_test, pred_for_acc_2, target_names=target_names))

[LibSVM]

/Users/nour/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Fit terminé
1.95
             precision    recall  f1-score   support

    class_1       0.04      0.54      0.07       399
    class_2       0.47      0.07      0.12      3178
    class_3       0.26      0.15      0.19      1561
    class_4       0.13      0.40      0.20       538
    class_5       0.76      0.62      0.68       565
    class_6       0.80      0.05      0.10      2884
    class_7       0.12      0.22      0.15       552
    class_8       0.36      0.15      0.21      1674
    class_9       0.20      0.20      0.20      1025

avg / total       0.45      0.16      0.17     12376



In [ ]:
1.77 100
1.76 1000
sur ordi Rayan 11min 5 sec fit 0.001 C=100 
1min50 fit extratree 0.47 sur TFIDF Dc dist 500 estimators cv 10